In [3]:
# Imports
import numpy as np
from matplotlib import rc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Formatting plots
plt.rcParams["axes.formatter.use_mathtext"] = True

sns.set(font="cmr10")
sns.set(font_scale = 2)
sns.set_theme(style="whitegrid", context="paper", font="cmr10")
sns.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 2.5})

In [5]:
# Utils
def save_figure(name):
    plt.savefig(f"exports/{name}.pdf", format="pdf", bbox_inches="tight")

def move_legend(ax, new_loc, **kws):
    old_legend = ax.legend_
    handles = old_legend.legend_handles
    labels = [t.get_text() for t in old_legend.get_texts()]
    title = old_legend.get_title().get_text()
    ax.legend(handles, labels, loc=new_loc, title=title, **kws)

In [6]:
# Data import

data = pd.read_csv('datasets/responses.csv')
data.sort_values(by='Responded At', inplace=True)
data.describe()

,[Participant] Age,[Participant] Reddit Usage,[Participant] Social Media Usage,[Participant] Fake News Ability,[Credibility] Credibility Rating,[Credibility] Topic familiarity,[Post] Title Sentiment,[Post] Award Count,[Post] Comments,[Post] Upvote Ratio,[Post] Upvotes
count,952.000000,952.000000,952.000000,952.000000,952.000000,952.000000,952.000000,952.0,952.000000,952.000000,952.000000
mean,39.191176,2.955882,4.382353,3.500000,3.139706,2.404412,-0.063990,0.0,252.053571,0.857941,3721.867647
std,15.470851,1.333917,1.164143,0.916033,1.147584,1.248891,0.162211,0.0,550.073099,0.154541,10038.217945
min,19.000000,1.000000,1.000000,2.000000,1.000000,1.000000,-0.416667,0.0,0.000000,0.500000,0.000000
25%,25.750000,2.000000,4.000000,3.000000,2.000000,1.000000,-0.128205,0.0,11.500000,0.850000,0.000000
50%,34.500000,3.000000,5.000000,4.000000,3.000000,2.000000,-0.060036,0.0,74.000000,0.920000,226.000000
75%,50.000000,4.000000,5.000000,4.000000,4.000000,3.000000,0.066667,0.0,179.000000,0.960000,2228.000000
max,90.000000,5.000000,5.000000,5.000000,5.000000,5.000000,0.200000,0.0,3121.000000,1.000000,54947.000000


In [7]:
# Columns

col_id = "Participant ID"

col_gender = "[Participant] Gender"
col_studies = "[Participant] Studies Level"
col_age = "[Participant] Age"
col_social_usage = "[Participant] Social Media Usage"
col_reddit_usage = "[Participant] Reddit Usage"
col_fake_news_ability = "[Participant] Fake News Ability"
col_reddit_as_news = "[Participant] Reddit As News Source"
col_reddit_credibility_evaluation = "[Participant] Reddit Credibility Evaluation"

col_credibility = "[Credibility] Credibility Rating"
col_would_upvote = "[Credibility] Would upvote"
col_would_upvote_reason = "[Credibility] Would upvote reason"
col_topic_familiarity = "[Credibility] Topic familiarity"
col_content_style = "[Credibility] Content Style"
col_content_style_effect = "[Credibility] Content Style Effect"

col_upvotes = "[Post] Upvotes"
col_ratio = "[Post] Upvote Ratio"
col_comments = "[Post] Comments"
# col_post_title = "[Post] Title"
# col_post_sub = "[Post] Subreddit"
# col_post_awards = "[Post] Award Count"